In [1]:
import numpy as np
from collections import deque
from copy import deepcopy

In [2]:
test = ['2413432311323',
        '3215453535623',
        '3255245654254',
        '3446585845452',
        '4546657867536',
        '1438598798454',
        '4457876987766',
        '3637877979653',
        '4654967986887',
        '4564679986453',
        '1224686865563',
        '2546548887735',
        '4322674655533']

test_path = ['2>>34^>>>1323',
             '32v>>>35v5623',
             '32552456v>>54',
             '3446585845v52',
             '4546657867v>6',
             '14385987984v4',
             '44578769877v6',
             '36378779796v>',
             '465496798688v',
             '456467998645v',
             '12246868655<v',
             '25465488877v5',
             '43226746555v>']

data = np.genfromtxt('day17_input.txt', dtype=str, delimiter='\n', comments=None)

In [33]:
def print_test_path(test_path):
    for i in range(0, len(test_path)):
        string = ''
        for j in range(0, len(test_path[i])):
            if test_path[i][j].isdigit():
                string += '.'
            else:
                string += test_path[i][j]
        print(string)

def gen_graph(data):
    graph = []
    for line in data:
        graph_line = []
        for c in line:
            graph_line.append(int(c))
        graph.append(graph_line)
    return np.array(graph)

def dijkstra(graph, start, end):
    # 0 >
    # 1 v
    # 2 <
    # 3 ^
    
    visited = {}
    routes = {}
    for i in range(0, len(graph)):
        for j in range(0, len(graph[i])):
            if i == 0 and j == 0:
                continue
            for k in range(0, 4):
                if (i == 0 and k == 1) or\
                   (i == len(graph)-1 and k == 3) or\
                   (j == 0 and k == 0) or\
                   (j == len(graph[i])-1 and k == 2):
                    continue
                visited[(i,j,k)] = [np.inf,0] #distance, chain
                routes[(i,j,k)] = []

    visited[(start[0],start[1],np.nan)] = [0,0]
    routes[(start[0],start[1],np.nan)] = [(start[0],start[1],0)]
    
    dxdy = [[0,1], [1,0], [0,-1], [-1,0]]
    
    run = True
    while run:
        has_update = False
        for vis in visited.keys():
                
            for i in range(0, len(dxdy)):
                xy = dxdy[i]
                nxt_pos = (vis[0]+xy[0], vis[1]+xy[1], i)
                if nxt_pos not in visited.keys():
                    continue
                
                #cannot backtrack
                if i == (vis[2]+2)%4:
                    continue
                    
                if vis[0] == 0 and vis[1] == 0:
                    chn = 1
                elif i == vis[2]:
                    chn = visited[vis][1]+1
                else:
                    chn = 1
                #cannot move more than 3 in one direction
                if chn > 3:
                    continue
                
                nxt_val = graph[nxt_pos[0],nxt_pos[1]] 
                nxt_dist = [visited[vis][0]+nxt_val, chn]
                
                rout = deepcopy(routes[vis])
                rout.append(nxt_pos)
                
                if np.isfinite(visited[nxt_pos][0]) == False or nxt_dist[0] < visited[nxt_pos][0]:
                    visited[nxt_pos] = nxt_dist
                    routes[nxt_pos] = rout
                    has_update = True
        
        if has_update == False:
            break
        
        run = False
        for vis in visited.keys():
            if np.isfinite(visited[vis][0]) == False:
                run = True
                break
                
    ends = []
    scores = []
    
    for i in range(0, 4):
        if (end[0],end[1],i) in visited.keys():
            ends.append((end[0],end[1],i))
            scores.append(visited[(end[0],end[1],i)][0])
            print((end[0],end[1],i), visited[(end[0],end[1],i)])
            
    min_dist = np.argmin(scores)
    
    return visited[ends[min_dist]][0], routes[ends[min_dist]]

def print_path(graph, route, dist):
    move = ['>', 'v', '<', '^']
    
    vals = []
    rt = []
    for i in range(0, len(graph)):
        string = ''
        for j in range(0, len(graph[i])):
            empty = True
            for k in range(0, 4):
                if (i,j,k) in route and (i!=0 or j !=0):
                    vals.append(graph[i,j])
                    string += move[k]
                    rt.append(string[-1])
                    empty = False
            if empty:
                string += '.'
        print(string)
    #print(vals)
    #print(rt)

def least_loss(data):
    graph = gen_graph(data)
    end = (len(graph)-1,len(graph[0])-1)
    dist, route = dijkstra(graph, (0,0), end)
    print_path(graph, route, dist)
    print(route)
    return dist

print_test_path(test_path)
print()

print(least_loss(test))
#print(least_loss(data))

.>>..^>>>....
..v>>>..v....
........v>>..
..........v..
..........v>.
...........v.
...........v.
...........v>
............v
............v
...........<v
...........v.
...........v>

(12, 12, 0) [109, 1]
(12, 12, 1) [105, 3]
.>>>..^>>>...
...v>>>..v...
.........v>..
..........v..
..........v>.
...........v>
............v
...........<v
...........v.
...........v>
............v
............v
............v
[(0, 0, 0), (0, 1, 0), (0, 2, 0), (0, 3, 0), (1, 3, 1), (1, 4, 0), (1, 5, 0), (1, 6, 0), (0, 6, 3), (0, 7, 0), (0, 8, 0), (0, 9, 0), (1, 9, 1), (2, 9, 1), (2, 10, 0), (3, 10, 1), (4, 10, 1), (4, 11, 0), (5, 11, 1), (5, 12, 0), (6, 12, 1), (7, 12, 1), (7, 11, 2), (8, 11, 1), (9, 11, 1), (9, 12, 0), (10, 12, 1), (11, 12, 1), (12, 12, 1)]
105


In [ ]:
#102